In [4]:
#Dependency
import tweepy
import json
import pandas as pd
from scipy.misc import imread
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib as mpl
#import csv
import matplotlib.pyplot as plt

import operator
from textblob import TextBlob
from textblob import Word
from textblob.sentiments import NaiveBayesAnalyzer

In [5]:
#################################################################
#Authentication
consumerKey = ' '
consumerSecret = ' '
accessToken = ' '
accessTokenSecret = ' '
auth = tweepy.OAuthHandler(consumerKey, consumerSecret)
auth.set_access_token(accessToken, accessTokenSecret)
#Connecting to Twitter API
api = tweepy.API(auth)

In [6]:
#################################################################
results = []
q = input("Enter Keyword/Hashtag: ")
for tweet in tweepy.Cursor (api.search, q , lang = "en").items(2000): 
    results.append(tweet)
    
#print (type(results))
#print (len(results))

Enter Keyword/Hashtag: #MAGA


TweepError: Twitter error response: status code = 429

In [ ]:
#################################################################
#Store tweets data in a dataframe

def tweets_df(results):
    id_list = [tweet.id for tweet  in results]
    data_set = pd.DataFrame(id_list, columns = ["id"])
    
    data_set["text"] = [tweet.text for tweet in results]
    data_set["created_at"] = [tweet.created_at for tweet in results]
    data_set["retweet_count"] = [tweet.retweet_count for tweet in results]
    data_set["user_screen_name"] = [tweet.author.screen_name for tweet in results]
    data_set["user_followers_count"] = [tweet.author.followers_count for tweet in results]
    data_set["user_location"] = [tweet.author.location for tweet in results]
    data_set["Hashtags"] = [tweet.entities.get('hashtags') for tweet in results]
    
    return data_set
data_set = tweets_df(results)


# Remove tweets with duplicate text

text = data_set["text"]

for i in range(0,len(text)):
    txt = ' '.join(word for word in text[i] .split() if not word.startswith('https:'))
    data_set.set_value(i, 'text2', txt)
    
data_set.drop_duplicates('text2', inplace=True)
data_set.reset_index(drop = True, inplace=True)
data_set.drop('text', axis = 1, inplace = True)
data_set.rename(columns={'text2': 'text'}, inplace=True)

In [ ]:
#################################################################
Htag_df = pd.DataFrame()
j = 0

for tweet in range(0,len(results)):
    hashtag = results[tweet].entities.get('hashtags')
    for i in range(0,len(hashtag)):
        Htag = hashtag[i]['text'] 
        Htag_df.set_value(j, 'Hashtag',Htag)
        j = j+1

In [ ]:
#Htag_df.head()

In [ ]:
#################################################################
# Join all the text from the 1000 tweets
Hashtag_Combined = " ".join(Htag_df['Hashtag'].values.astype(str))

no_included = " ".join([word for word in Hashtag_Combined.split()
                               if word != q                                             
                                ])



#Create a Word Cloud
wc = WordCloud(background_color="white", stopwords=STOPWORDS)
wc.generate(no_included)
fig = plt.figure(figsize=(10,8))
plt.imshow(wc)
plt.axis("off")
#plt.savefig('path', dpi=300)
plt.show(fig)
